In [4]:
from enum import Enum
import itertools as it
import copy
from multiprocessing import Pool, cpu_count

with open('input.txt', 'r') as file:
    input = file.read()

test_input = """\
190: 10 19
3267: 81 40 27
83: 17 5
156: 15 6
7290: 6 8 6 15
161011: 16 10 13
192: 17 8 14
21037: 9 7 18 13
292: 11 6 16 20
"""

In [54]:

class ElephantPlay():
    def __init__(self,txt):
        self.equations = [(int(row.split(":")[0]),
                          [x for x in row.split(": ")[1].split(" ")]
                          ) for row in txt.splitlines()]
        self.operators = ["+","*"]

    def build_nested_eq(self,values,operators):
        if len(values) == 1: return values[0] # base recursion
        else: # recursively call this function until we reach base
            inner_formula = f"({values[0]}{operators[0]}{values[1]})"
            return self.build_nested_eq([inner_formula] + values[2:],operators[1:])

    def check_eqs(self,eq):
        c = 0
        for answer, values in self.equations:
            for opers in it.product(self.operators,repeat=len(values)-1):
                # print(opers)
                formula = self.build_nested_eq(values,opers) #"".join(o+v for o,v in zip([""]+list(opers),values))
                # print(formula)
                if eval(formula) == answer:
                    # print(answer)
                    c+=answer
                    break
        return c

res = ElephantPlay(test_input)
res.check_eqs(res.equations[0])

3749

### Part 2
Struggled for a bit because I didn't want to re-write the eval logic. Then I realized we are all better if I did even though I really liked my previous recursive method. Oh well

In [13]:

class ElephantPlay():
    def __init__(self,txt):
        self.equations = [(int(row.split(":")[0]),
                          [x for x in row.split(": ")[1].split(" ")]
                          ) for row in txt.splitlines()]
        self.operators = ["+","*","||"]

    def build_nested_eq(self,values,operators):
        if len(values) == 1: return eval(values[0]) # base recursion
        else: # recursively call this function until we reach base
            inner_formula = f"({values[0]}{operators[0]}{values[1]})"
            return self.build_nested_eq([inner_formula] + values[2:],operators[1:])
    
    def custom_eval(self,values,opers):
        # print(values,opers)
        i = 1
        res = int(values[0])
        # print(res)
        for op in opers:
            # print(op,"... =")
            match op:
                case "+":
                    res+=int(values[i])
                case "*":
                    res*=int(values[i])
                case "||":
                    res=int(str(res)+str(values[i]))
            i+=1
            # print(res)
        return res
    
    def check_eq(self,eq):
        answer, values = eq
        for opers in it.product(self.operators,repeat=len(values)-1):
            if self.custom_eval(values,opers) == answer:
                return answer
        return 0

    def check_eqs(self):
        # Changing to multiprocessing:
        with Pool(processes=cpu_count()) as pool:
            results = pool.map(self.check_eq,self.equations)
            # working with a list, this maps an equation to every item of the list
            return sum(results)
        # c = 0
        # for answer, values in self.equations:
        #     for opers in it.product(self.operators,repeat=len(values)-1):
        #         if self.custom_eval(values,opers) == answer:
        #             # print(answer)
        #             c+=answer
        #             break
        # return c
    

res = ElephantPlay(test_input)
res.check_eqs()

11387

takeaway:
- multiprocessing reduced a 5min+ process into a 20s one. awesome
- it's still good to figure out the input set first before using multiprocessing to reduce load